# Importing Required Libraries

In [ ]:
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import AdamW

# Helper Functions

In [ ]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 16
DATASET_DIR = '' # Change this to the path of the dataset
CLASSES_LIST = ['Brawl', 'Peace']

In [ ]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)

    video_reader.release()
    return frames_list

In [ ]:
def create_dataset(DATASET_DIR):
    features = []
    labels = []
    video_files_paths = []

    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            frames = frames_extraction(video_file_path)
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)
    
    return features, labels, video_files_paths

# Extracting Features

In [ ]:
features, labels, video_files_paths = create_dataset(DATASET_DIR=DATASET_DIR)
one_hot_encoded_labels = to_categorical(labels)
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = 42)

# Model

In [ ]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

def build_optimized_slowfast_improved(input_shape, num_classes, alpha=4):
    inputs = layers.Input(shape=input_shape)
    
    slow = layers.Lambda(lambda x: x[:, ::alpha, :, :, :])(inputs)
    slow = layers.Conv3D(32, (1, 7, 7), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(slow)
    slow = layers.BatchNormalization()(slow)
    slow = layers.MaxPooling3D((1, 2, 2), padding='same')(slow)
    
    fast = inputs
    fast = layers.Conv3D(16, (1, 5, 5), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(fast)
    fast = layers.BatchNormalization()(fast)
    fast = layers.MaxPooling3D((1, 2, 2), padding='same')(fast)
    fast = layers.MaxPooling3D(pool_size=(alpha, 1, 1), padding='same')(fast)
    
    slow = layers.Conv3D(16, (1, 1, 1), activation='relu', padding='same')(slow)
    fast = layers.Conv3D(16, (1, 1, 1), activation='relu', padding='same')(fast)
    
    fusion = layers.Concatenate(axis=-1)([slow, fast])
    
    x = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(fusion)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling3D((2, 2, 2), padding='same')(x)
    x = layers.SpatialDropout3D(0.2)(x)
    
    shortcut = x
    x = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv3D(64, (3, 3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    
    x = layers.MaxPooling3D((2, 2, 2), padding='same')(x)
    
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer=AdamW(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

frame_count = 16
height, width, channels = 64, 64, 3
num_classes = 2

model = build_optimized_slowfast_improved((frame_count, height, width, channels), num_classes)
model.summary()

In [ ]:
history = model.fit(features_train, labels_train, epochs=100, batch_size=8, verbose=1, validation_split=0.2)


In [ ]:
pred = model.predict(features_test)
pred = np.argmax(pred, axis = 1)
labels_test = np.argmax(labels_test, axis = 1)
print(classification_report(labels_test, pred, target_names = CLASSES_LIST))

# Final Predictions

In [ ]:
PREDICT_DATASET_DIR = '' # Change this to the path of the predict dataset
predict_features, predict_labels, predict_video_files_paths = create_dataset(DATASET_DIR=PREDICT_DATASET_DIR)
predict_labels = to_categorical(predict_labels)

pred = model.predict(predict_features)
predicted_classes = np.argmax(pred, axis = 1)
labels
print(classification_report(labels, predicted_classes, target_names = CLASSES_LIST))